In [1]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

<hr style="border-top: 5px solid black;">

<h1>Import Packages</h1>

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

<hr style="border-top: 5px solid black;">

<h1>Import Data</h1>

In [3]:
file = "authors_data.xlsx"
df = pd.read_excel(file, sheet_name=0)

<hr style="border-top: 5px solid black;">

<h1>Inspect Data</h1>

In [4]:
df.head()

,ID,Author,Degree,Degree Discipline,Title,Date,Has_Contributors,Co-ID,Contributor Name
0,ID-121,Tori Marsh,MPH,Drug pricing and savings,‘Vaccine Deserts’ Threaten to Prolong COVID-19...,2021-01-14,Yes,ID-917,Sara Kim
1,ID-121,Tori Marsh,MPH,Drug pricing and savings,‘Vaccine Deserts’ Threaten to Prolong COVID-19...,2021-01-14,Yes,ID-506,Amanda Nguyen
2,ID-121,Tori Marsh,MPH,Drug pricing and savings,‘Vaccine Deserts’ Threaten to Prolong COVID-19...,2021-01-14,Yes,ID-204,Jeroen van Meijgaard
3,ID-121,Tori Marsh,MPH,Drug pricing and savings,800+ Drugs Became More Expensive This January ...,2021-02-02,Yes,ID-204,Jeroen van Meijgaard
4,ID-121,Tori Marsh,MPH,Drug pricing and savings,Anxiety and Depression Medication Prices Have ...,2019-09-10,Yes,ID-204,Jeroen van Meijgaard


In [5]:
authors = df.groupby(by=["ID", "Author"])["Title"].count().to_frame()
authors

,,Title
ID,Author,
ID-121,Tori Marsh,69
ID-204,Jeroen van Meijgaard,9
ID-400,Diane Li,15
ID-471,Lauren Chase,42
ID-506,Amanda Nguyen,21
ID-610,Sasha Guttentag,10


<hr style="border-top: 5px solid black;">

<h1>Text K-means Clustering</h1>

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(stop_words={'english'})

In [ ]:
titles = df.Title.unique()

In [ ]:
X = vectorizer.fit_transform(titles)

In [ ]:
print(vectorizer.get_feature_names())
print(X.shape)

In [ ]:
print(X[0,])

In [ ]:
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans

In [ ]:
Sum_of_squared_distances = []
K = range(2,10)
for k in K:
    km = KMeans(n_clusters=k, max_iter=200, n_init=10)
    km = km.fit(X)
    Sum_of_squared_distances.append(km.inertia_)

plt.plot(K, Sum_of_squared_distances, 'bx-')
plt.xlabel('k')
plt.ylabel('Sum_of_squared_distances')
plt.title('Elbow Method For Optimal k')
plt.show()

In [ ]:
true_k = 5
model = KMeans(n_clusters=true_k, init='k-means++', max_iter=200, n_init=10)
model.fit(X)
labels=model.labels_
articles_cl=pd.DataFrame(list(zip(titles,labels)),columns=['titles','cluster'])
print(articles_cl.sort_values(by=['cluster']))

In [ ]:
from wordcloud import WordCloud
result={'cluster':labels,'authors':titles}
result=pd.DataFrame(result)
for k in range(0,true_k):
    s=result[result.cluster==k]
    text=s['authors'].str.cat(sep=' ')
    text=text.lower()
    text=' '.join([word for word in text.split()])
    wordcloud = WordCloud(max_font_size=50, max_words=10, background_color="white").generate(text)
    print('Cluster: {}'.format(k))
    print('Titles')
    titles=articles_cl[articles_cl.cluster==k]['titles']
    print(titles.to_string(index=False))
    plt.figure()
    plt.imshow(wordcloud, interpolation="bilinear")
    plt.axis("off")
    plt.show()

<hr style="border-top: 5px solid black;">

<h1>Word Clouds from Dataframe</h1>

In [6]:
# Visualize wordcloud
from wordcloud import WordCloud, STOPWORDS

# Import Natural Language Toolkit
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk import word_tokenize
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/trinidadcisneros/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/trinidadcisneros/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/trinidadcisneros/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

<hr style="border-top: 2px solid black;">

# Group All Titles by Author ID

 - Drop Duplicates
 - Group by and combine titles by author

In [7]:
author_titles_df = df.drop_duplicates(subset=['Title'], keep='last')
author_titles_df.head()

,ID,Author,Degree,Degree Discipline,Title,Date,Has_Contributors,Co-ID,Contributor Name
2,ID-121,Tori Marsh,MPH,Drug pricing and savings,‘Vaccine Deserts’ Threaten to Prolong COVID-19...,2021-01-14,Yes,ID-204,Jeroen van Meijgaard
3,ID-121,Tori Marsh,MPH,Drug pricing and savings,800+ Drugs Became More Expensive This January ...,2021-02-02,Yes,ID-204,Jeroen van Meijgaard
6,ID-121,Tori Marsh,MPH,Drug pricing and savings,Anxiety and Depression Medication Prices Have ...,2019-09-10,Yes,ID-262,Clement B Feyt
8,ID-121,Tori Marsh,MPH,Drug pricing and savings,Can’t Access Generic Humalog? There’s an Even ...,2019-08-26,Yes,ID-204,Jeroen van Meijgaard
9,ID-121,Tori Marsh,MPH,Drug pricing and savings,COVID-19 Causes a Drop in Fills for HIV Preven...,2020-10-20,Yes,ID-471,Lauren Chase


In [8]:
author_titles = author_titles_df.groupby(['ID', 'Author'], as_index = False).agg({'Title': ' '.join})
author_titles

,ID,Author,Title
0,ID-121,Tori Marsh,‘Vaccine Deserts’ Threaten to Prolong COVID-19...
1,ID-204,Jeroen van Meijgaard,Drug Makers Keep Promises to Limit Price Incre...
2,ID-400,Diane Li,A Look at Medicare Part D Plans in 2020: Highe...
3,ID-471,Lauren Chase,A Guide to Biosimilar Prices: How Much They Co...
4,ID-506,Amanda Nguyen,25 Million Americans Have Lost Their Health In...
5,ID-610,Sasha Guttentag,Almost 60% of the Public Plans to Get the COVI...


<hr style="border-top: 2px solid black;">

# Word Tokenization

 - Action: Return a tokenized copy of string
 - word_tokenize(string)
 - Documentation: https://www.nltk.org/_modules/nltk/tokenize.html

In [9]:
# Create a list of lists that contain tokens for each word in each list
word_tokens = [word_tokenize(text) for text in author_titles.Title]

In [10]:
# Token processing, remove non alpha numeric tokens from each list
cleaned_tokens = [[word for word in item if word.isalnum()] for item in word_tokens]

In [11]:
# Add results to dataframe
author_titles["tokens"] = cleaned_tokens

<hr style="border-top: 2px solid black;">

# Word Token Lemmatizing

 - Action: Lemmatization is the process of converting a word to its base form.
 - WNlemmatizer = WordNetLemmatizer()
 - lemmanized = WNlemmatizer.lemmatize(token)
 - Documentation: https://www.nltk.org/_modules/nltk/stem/wordnet.html

In [12]:
lemm_list = []
WNlemmatizer = WordNetLemmatizer()
for index, series in author_titles["tokens"].iteritems():
    lemm_list.append([WNlemmatizer.lemmatize(token).lower() for token in series])

In [13]:
author_titles["lemm_tokens"] = lemm_list

<hr style="border-top: 2px solid black;">

# WordCloud Analysis

 - Reveals essential
 - Provides an overall sense of the text
 - Easy to grasp and engaging
 - wordcloud = WordCloud().generate(text)
 - Documentation: https://amueller.github.io/word_cloud/

## Step 1: Filter Text Using Custom Stop Word List

 - Combine stop words from wordcloud, nltk, and custom list

In [14]:
# Wordcloud stopwords
wc_sw = STOPWORDS
custom_sps = set(['made', 'nothing','able', 'given', 'wish', 'willing', 'wa', 'due', 'ha', 'did','etc', 'use', 'really', 'felt', 'personally', 'also', 'thing', 'well', 'little', 'got', 'one', 'lot', 'way', 'jus', 'sure'])
cust_nltk_sw = set(stopwords.words('english')).union(custom_sps).union(wc_sw)
print(f'The number of stop words in custom list is: {len(cust_nltk_sw)}')

The number of stop words in custom list is: 250


## Step 2: Visualize WordCloud for all Title by Authors in dataframe

### Step 2a: Create function that converts tokenized text into a single string

In [ ]:
# This will be used in the wordcloud
def convert_text(series):
    np_array = series.array
    text = ' '.join(str(v) for v in np_array[0])
    return text

### Step 2b: Create Wordcloud PNG files for each Author ID 

In [ ]:
author_ids = author_titles.ID
nbr_features = len(author_titles.ID)

# for i in range(0,nbr_features):
for id_num, num in zip(author_ids, range(0,nbr_features)):
    fig = plt.figure(figsize=(30,15))
    fig.subplots_adjust(hspace=1, wspace=0.2)

    df0=author_titles[author_titles['ID']==id_num]['lemm_tokens']
    text = convert_text(df0)
    cloud = WordCloud(background_color="white", max_words=10, stopwords=cust_nltk_sw, collocations=False).generate(text)

    plt.imshow(cloud, interpolation='bilinear')
    plt.title(id_num, fontsize = 20)
    plt.axis('off')
    plt.savefig(f"{id_num}_wordcloud.png", bbox_inches='tight', dpi=300)

### Step 2c: Create figure that contains all wordclouds, each as a subplot

In [ ]:
author_ids = author_titles.ID
nbr_features = len(author_titles.ID)
fig = plt.figure(figsize=(30,15))
fig.subplots_adjust(hspace=1, wspace=0.2)

for id_num, num in zip(author_ids, range(1,nbr_features)):
    df0=author_titles[author_titles['ID']==id_num]['lemm_tokens']
    text = convert_text(df0)
    cloud = WordCloud(background_color="white", max_words=10, stopwords=cust_nltk_sw, collocations=False).generate(text)
    ax = fig.add_subplot(4,3, num)
    ax.imshow(cloud, interpolation='bilinear')
    ax.set_title(id_num)
    ax.axis('off')

plt.tight_layout()
plt.show()
plt.savefig(f"all_wordcloud.png", bbox_inches='tight', dpi=300)

## Step 3: Visualize WordCloud for all Titles

 - Combine All Titles
 - Generate WordCloud

In [ ]:
titles = ' '.join(author_titles_df["Title"])
title_list = []
title_list.append(titles)
title_df = pd.DataFrame.from_dict({'ID': 'ID-All', 'Title' : title_list})
title_df

In [ ]:
# Create a list of lists that contain tokens for each word in each list
word_tokens = [word_tokenize(text) for text in title_df.Title]

# Token processing, remove non alpha numeric tokens from each list
cleaned_tokens = [[word for word in item if word.isalnum()] for item in word_tokens]
# Add results to dataframe
title_df["tokens"] = cleaned_tokens

lemm_list = []
WNlemmatizer = WordNetLemmatizer()
for index, series in title_df["tokens"].iteritems():
    lemm_list.append([WNlemmatizer.lemmatize(token).lower() for token in series])

title_df["lemm_tokens"] = lemm_list


# Wordcloud stopwords
wc_sw = STOPWORDS
custom_sps = set(['made', 'nothing','able', 'given', 'wish', 'willing', 'wa', 'due', 'ha', 'did','etc', 'use', 'really', 'felt', 'personally', 'also', 'thing', 'well', 'little', 'got', 'one', 'lot', 'way', 'jus', 'sure'])
cust_nltk_sw = set(stopwords.words('english')).union(custom_sps).union(wc_sw)
print(f'The number of stop words in custom list is: {len(cust_nltk_sw)}')

title_df

In [ ]:
author_id = title_df.ID
nbr_features = len(title_df.ID)

fig = plt.figure(figsize=(30,15))

df0=title_df['lemm_tokens']
text = convert_text(df0)
cloud = WordCloud(background_color="white", max_words=10, stopwords=cust_nltk_sw, collocations=False).generate(text)

plt.imshow(cloud, interpolation='bilinear')
plt.title(title_df.ID.values, fontsize = 20)
plt.axis('off')
plt.savefig(f"{author_id}_wordcloud.png", bbox_inches='tight', dpi=300)

<hr style="border-top: 5px solid black;">

<h1>Create new DataFrame for Displaying WordCloud Images</h1>

 - Concatenate author_titles and titles
 - Add column for url
 - Add url
 - Export as CSV for tableau

In [ ]:
frames = [title_df, author_titles]
word_cloud_df = pd.concat(frames, ignore_index=True)
word_cloud_df

In [ ]:
url_lists = ['https://i.imgur.com/NGDC7RA.png','https://i.imgur.com/HE8dkGa.png','https://i.imgur.com/EtQDWsW.png', 'https://i.imgur.com/bxm8T6x.png', 'https://i.imgur.com/2Gch9y0.png','https://imgur.com/YlKbGz8', 'https://i.imgur.com/UzCst7H.png']
word_cloud_df["urls"] = url_lists

In [ ]:
word_cloud_df.to_csv("id_wordclouds.csv", index_label=False)

<hr style="border-top: 5px solid black;">

In [15]:
<h1>Network Analysis</h1>

SyntaxError: invalid syntax (3881721734.py, line 1)

In [ ]:
# libraries
import pandas as pd
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
 
# Build a dataframe with your connections
df = pd.DataFrame({ 'from':['A', 'B', 'C','A'], 'to':['D', 'A', 'E','C']})
 
# Build your graph
G=nx.from_pandas_edgelist(df, 'from', 'to')
 
# Chart with Custom edges:
nx.draw(G, with_labels=True, width=5, edge_color="skyblue", style="solid")

In [ ]:
df.head()

In [ ]:
# Build your graph
G=nx.from_pandas_edgelist(df, 'from', 'to')
 
# Chart with Custom edges:
nx.draw(G, with_labels=True, width=5, edge_color="skyblue", style="solid")